**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The parameter epsilon controls the behavior of the agent in terms exploration vs exploitation. It means the bigger $\epsilon$ gets, the more random the agent acts. This allows to explore more states, and thus not possibly miss a bigger reward with a riskier policy. It also allow the model to train on unseen states. Conversely, when $\epsilon$ is low, the agent relies on what it already learned, and acts according to what is going to maximize its reward, given the positions it already visited.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            # Go right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            # Go left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            # Go up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            # Go down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array indicates where the agent is on the board. It takes value 0 where the agent is not, and 1 where it is. The special value -1 is used to pad the forbidden coordinates outside of the board.
The board array contains the information about the reward: it equals 0.5 in x,y if there is cheese, -1 if there is poison and 0 if there is nothing in particular. The cheese and poison cells are chosen randomly using a Bernoulli random variable, controlled by the temperature parameter.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            # loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        ##### FILL IN HERE
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=10, max_time=100,temperature=0.1)

# Initialize the agent!
agent = RandomAgent()

epochs_test = 5

from IPython.display import Video

test(agent,env,epochs_test,prefix='random')
# Video('random0.mp4')

Win/lose count 3.0/2.0. Average score (1.0)
Win/lose count 2.0/2.0. Average score (0.5)
Win/lose count 3.0/3.0. Average score (0.3333333333333333)
Win/lose count 2.0/3.0. Average score (0.0)
Win/lose count 1.0/3.0. Average score (-0.4)
Final score: -0.4


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Let's show that
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

By Q's definition, we have:
\begin{align*}
Q^{\pi}(s,a) & = E_{p^{\pi}}[\sum_{0 \leq t} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
 & = E_{p^{\pi}}[r(s_0, a_0) + \sum_{1 \leq t} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
 & = E_{p^{\pi}}[r(s,a) + \sum_{0 \leq t} \gamma^{t+1} r(s_{t+1}, a_{t+1}) | s_0=s, a_0=a] \\
 & = E_{p^{\pi}}[r(s,a) + \gamma \sum_{0 \leq t} \gamma^{t} r(s_{t+1}, a_{t+1}) | s_0=s, a_0=a] \\
 & = E_{(s',a') \sim p(.|a,s)} \big [ E_{p^{\pi}}[r(s,a) + \gamma \sum_{0 \leq t} \gamma^{t} r(s_{t+1}, a_{t+1}) | s_0=s, a_0=a] \big ] \\
 & = E_{(s',a') \sim p(.|a,s)} \big [ r(s,a) + \gamma E_{p^{\pi}}[\sum_{0 \leq t} \gamma^{t} r(s_{t+1}, a_{t+1}) | s_0=s, a_0=a] \big ] \\
 & = E_{(s',a') \sim p(.|a,s)} \big [ r(s,a) + \gamma E_{p^{\pi}}[\sum_{0 \leq t} \gamma^{t} r(s_{t+1}, a_{t+1}) | s_1=s', a_1=a'] \big ] \\
 & = E_{(s',a') \sim p(.|a,s)} \big [ r(s,a) + \gamma Q^{\pi} (s',a') \big ] \\
\end{align*}

Now let's justify:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

It is the Bellman's equation. It is the same reasonning as on a graph. The value of Q is maximal, when, given a state and an action chosen following the optimal policy (first term), we choose the action that maximizes the second term ie the Q function with state s' resulting from starting with state s and taking action a.

Given these derivations, it makes sense to choose the following loss
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation*}
because it forces the network to try and close the gap between what we know is the true value for Q when transitioning from state s to s', and the value we value we predict without Bellman's equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.index = 0
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.index] = m
        self.index = (self.index + 1)%self.max_memory

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # decreasing epsilon-greedy policy
        agent.set_epsilon(agent.epsilon*0.95)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.9
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]), batch_size=1))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # EDIT STARTS HERE
        
        for i in range(self.batch_size):
            state, new_state, action, reward, game_over = self.memory.random_access()
            input_states[i,:,:,:] = state

            # if done, make our target reward
            target = reward

            if not game_over:
                # predict the future discounted reward
                target = reward + self.discount * \
                    np.amax(self.model.predict(np.array([new_state]), batch_size=1))

            # make the agent to approximately map
            # the current state to future discounted reward
            # We'll call that target_f
            target_q[i,:] = self.model.predict(np.array([state]), batch_size=1)
            target_q[i,action] = target
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # NN Model
        model = keras.Sequential()
        model.add(Flatten(input_shape=(5,5, self.n_state)))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4, activation='linear'))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=10 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train170.mp4'))

Epoch 000/200 | Loss 0.0267 | Win/lose count 12.5/9.0 (3.5)
Epoch 001/200 | Loss 0.0129 | Win/lose count 5.5/5.0 (0.5)
Epoch 002/200 | Loss 0.0247 | Win/lose count 7.0/13.0 (-6.0)
Epoch 003/200 | Loss 0.0248 | Win/lose count 8.0/6.0 (2.0)
Epoch 004/200 | Loss 0.0199 | Win/lose count 6.5/7.0 (-0.5)
Epoch 005/200 | Loss 0.0324 | Win/lose count 10.5/6.0 (4.5)
Epoch 006/200 | Loss 0.0066 | Win/lose count 7.0/1.0 (6.0)
Epoch 007/200 | Loss 0.0159 | Win/lose count 8.0/12.0 (-4.0)
Epoch 008/200 | Loss 0.0041 | Win/lose count 6.0/6.0 (0.0)
Epoch 009/200 | Loss 0.0076 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/200 | Loss 0.0063 | Win/lose count 4.0/4.0 (0.0)
Epoch 011/200 | Loss 0.0032 | Win/lose count 6.5/6.0 (0.5)
Epoch 012/200 | Loss 0.0075 | Win/lose count 1.0/4.0 (-3.0)
Epoch 013/200 | Loss 0.0070 | Win/lose count 9.0/9.0 (0.0)
Epoch 014/200 | Loss 0.0060 | Win/lose count 2.0/4.0 (-2.0)
Epoch 015/200 | Loss 0.0250 | Win/lose count 6.5/8.0 (-1.5)
Epoch 016/200 | Loss 0.0185 | Win/lose count 4

Epoch 141/200 | Loss 0.0004 | Win/lose count 8.0/1.0 (7.0)
Epoch 142/200 | Loss 0.0011 | Win/lose count 1.5/1.0 (0.5)
Epoch 143/200 | Loss 0.0012 | Win/lose count 5.0/0 (5.0)
Epoch 144/200 | Loss 0.0025 | Win/lose count 6.5/1.0 (5.5)
Epoch 145/200 | Loss 0.0026 | Win/lose count 2.5/1.0 (1.5)
Epoch 146/200 | Loss 0.0053 | Win/lose count 6.0/0 (6.0)
Epoch 147/200 | Loss 0.0014 | Win/lose count 6.5/0 (6.5)
Epoch 148/200 | Loss 0.0038 | Win/lose count 1.0/1.0 (0.0)
Epoch 149/200 | Loss 0.0028 | Win/lose count 2.0/0 (2.0)
Epoch 150/200 | Loss 0.0007 | Win/lose count 5.0/1.0 (4.0)
Epoch 151/200 | Loss 0.0005 | Win/lose count 12.5/1.0 (11.5)
Epoch 152/200 | Loss 0.0049 | Win/lose count 4.0/0 (4.0)
Epoch 153/200 | Loss 0.0037 | Win/lose count 5.0/0 (5.0)
Epoch 154/200 | Loss 0.0088 | Win/lose count 2.0/0 (2.0)
Epoch 155/200 | Loss 0.0067 | Win/lose count 5.5/4.0 (1.5)
Epoch 156/200 | Loss 0.0047 | Win/lose count 4.5/0 (4.5)
Epoch 157/200 | Loss 0.0008 | Win/lose count 7.5/2.0 (5.5)
Epoch 158/2

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # NN Model
        model = keras.Sequential()
        # model.add(Flatten(input_shape=(5,5, self.n_state)))
        model.add(Conv2D(16, (3,3), activation='relu'))
        model.add(Conv2D(32, (3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=10 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.6, memory_size=2000, batch_size = 32)

In [15]:
train(agent,env,epochs_train,prefix='cnn_train')
    HTML(display_videos('cnn_train180.mp4'))

Epoch 000/200 | Loss 0.0082 | Win/lose count 5.0/11.0 (-6.0)
Epoch 001/200 | Loss 0.0112 | Win/lose count 6.5/16.0 (-9.5)
Epoch 002/200 | Loss 0.0094 | Win/lose count 5.0/7.0 (-2.0)
Epoch 003/200 | Loss 0.0106 | Win/lose count 6.0/12.0 (-6.0)
Epoch 004/200 | Loss 0.0087 | Win/lose count 14.0/9.0 (5.0)
Epoch 005/200 | Loss 0.0054 | Win/lose count 7.5/7.0 (0.5)
Epoch 006/200 | Loss 0.0243 | Win/lose count 6.5/9.0 (-2.5)
Epoch 007/200 | Loss 0.0026 | Win/lose count 11.5/7.0 (4.5)
Epoch 008/200 | Loss 0.0133 | Win/lose count 8.5/7.0 (1.5)
Epoch 009/200 | Loss 0.0104 | Win/lose count 9.5/4.0 (5.5)
Epoch 010/200 | Loss 0.0037 | Win/lose count 10.0/4.0 (6.0)
Epoch 011/200 | Loss 0.0118 | Win/lose count 10.0/5.0 (5.0)
Epoch 012/200 | Loss 0.0148 | Win/lose count 17.5/8.0 (9.5)
Epoch 013/200 | Loss 0.0060 | Win/lose count 15.5/6.0 (9.5)
Epoch 014/200 | Loss 0.0062 | Win/lose count 10.5/5.0 (5.5)
Epoch 015/200 | Loss 0.0079 | Win/lose count 17.5/4.0 (13.5)
Epoch 016/200 | Loss 0.0111 | Win/lose 

Epoch 137/200 | Loss 0.0125 | Win/lose count 18.5/1.0 (17.5)
Epoch 138/200 | Loss 0.0132 | Win/lose count 21.5/1.0 (20.5)
Epoch 139/200 | Loss 0.0129 | Win/lose count 16.5/0 (16.5)
Epoch 140/200 | Loss 0.0143 | Win/lose count 23.5/2.0 (21.5)
Epoch 141/200 | Loss 0.0107 | Win/lose count 15.5/0 (15.5)
Epoch 142/200 | Loss 0.0251 | Win/lose count 23.5/1.0 (22.5)
Epoch 143/200 | Loss 0.0110 | Win/lose count 18.5/1.0 (17.5)
Epoch 144/200 | Loss 0.0112 | Win/lose count 16.5/0 (16.5)
Epoch 145/200 | Loss 0.0065 | Win/lose count 14.5/0 (14.5)
Epoch 146/200 | Loss 0.0101 | Win/lose count 16.5/1.0 (15.5)
Epoch 147/200 | Loss 0.0179 | Win/lose count 18.0/1.0 (17.0)
Epoch 148/200 | Loss 0.0116 | Win/lose count 8.5/0 (8.5)
Epoch 149/200 | Loss 0.0156 | Win/lose count 13.5/0 (13.5)
Epoch 150/200 | Loss 0.0184 | Win/lose count 7.0/0 (7.0)
Epoch 151/200 | Loss 0.0058 | Win/lose count 22.0/0 (22.0)
Epoch 152/200 | Loss 0.0123 | Win/lose count 18.0/1.0 (17.0)
Epoch 153/200 | Loss 0.0104 | Win/lose count

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

After training the FCN and CNN, we make the following observations:
* Both networks tend to get blocked in cycles. This tendency decreases as the temperature increase. This is problably because when the board is full of cheese, the state seen by the agent keeps on evolving, and thus the agent acts differently every time.
* Even though the agent could visit almost each position during game time, it ends up going on the same cells very often, so it doesn't get a chance to eat every piece of cheese.
* The FCN is consistently doing worse than the CNN. One interpretation could be the following: we train both networks with similar caracteristics and parameters, but the CNN is a FCN with lots of shared weights so it leverages invariances in the problem in order to learn faster.

In [16]:
temperatures = [0.05, 0.1, 0.3, 0.5]
for temp in temperatures:
    env = Environment(grid_size=size, max_time=T, temperature=temp)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN with T =', temp)
    test(agent_cnn,env,epochs_test,prefix='cnn_test_'+str(temp*100))
    print('Test of the FC with T =', temp)
    test(agent_fc,env,epochs_test,prefix='fc_test'+str(temp*100))

Test of the CNN with T = 0.05
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 3.5/1.0. Average score (2.0)
Win/lose count 2.5/0. Average score (2.1666666666666665)
Win/lose count 2.5/1.0. Average score (2.0)
Win/lose count 3.5/2.0. Average score (1.9)
Win/lose count 3.5/2.0. Average score (1.8333333333333333)
Win/lose count 1.5/0. Average score (1.7857142857142858)
Win/lose count 2.0/0. Average score (1.8125)
Win/lose count 1.5/0. Average score (1.7777777777777777)
Win/lose count 5.5/1.0. Average score (2.05)
Final score: 2.05
Test of the FC with T = 0.05
Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 1.0/2.0. Average score (-0.6666666666666666)
Win/lose count 2.5/1.0. Average score (-0.125)
Win/lose count 1.0/1.0. Average score (-0.1)
Win/lose count 1.0/0. Average score (0.08333333333333333)
Win/lose count 3.0/1.0. Average score (0.35714285714285715)
Win/lose count 2.0/1.0. Average score (0.4375)
Win/lose count 2.0/0

In [17]:
HTML(display_videos('cnn_test_30.04.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # decreasing epsilon-greedy policy
        agent.set_epsilon(agent.epsilon*0.95)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


class EnvironmentExploring(Environment):
    def __init__(self, *args, grid_size=10, train=False, **kwargs):
        super(EnvironmentExploring, self).__init__(*args,grid_size=grid_size,**kwargs)

        self.visited = np.zeros((grid_size,grid_size))
    
    def act(self, action, train=False):
        state, reward, game_over = super().act(action)
        
        if train:
            reward -= 0.05*self.visited[self.x, self.y]
        
        self.visited[self.x, self.y] += 1
        self.visited = np.clip(self.visited, 0, 3)
        
        # We decrease the visited states so that the agent can go back on its
        # track when it has finished visiting a specific area.
        self.visited *= 0.8
        
        state = np.concatenate((
            state,
            self.visited.reshape(
                self.grid_size, self.grid_size,1
            )[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :],
        ), axis=2)
        
        return state, reward, game_over
    
    def reset(self):
        state = super().reset()
        
        self.visited = np.zeros((self.grid_size,self.grid_size))
        
        state = np.concatenate((
            state,
            self.visited.reshape(
                self.grid_size, self.grid_size,1
            )[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :],
        ), axis=2)
        
        return state

In [19]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=10 # set small when debugging

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 16, n_state=3)

In [20]:
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore_80.mp4'))

Epoch 000/100 | Loss 0.0093 | Win/lose count 5.0/12.736957752411184 (-7.736957752411184)
Epoch 001/100 | Loss 0.0358 | Win/lose count 4.0/12.374236208464772 (-8.374236208464772)
Epoch 002/100 | Loss 0.0089 | Win/lose count 6.5/16.1718220479036 (-9.6718220479036)
Epoch 003/100 | Loss 0.0081 | Win/lose count 5.0/10.756564380170092 (-5.756564380170092)
Epoch 004/100 | Loss 0.0022 | Win/lose count 6.5/14.918097118448303 (-8.418097118448303)
Epoch 005/100 | Loss 0.0038 | Win/lose count 7.0/12.13595419830726 (-5.13595419830726)
Epoch 006/100 | Loss 0.0170 | Win/lose count 9.0/11.55567611498693 (-2.5556761149869303)
Epoch 007/100 | Loss 0.0069 | Win/lose count 8.0/8.260590854221055 (-0.2605908542210553)
Epoch 008/100 | Loss 0.0038 | Win/lose count 8.5/16.227750917646436 (-7.727750917646436)
Epoch 009/100 | Loss 0.0025 | Win/lose count 20.0/7.14377526139858 (12.85622473860142)
Epoch 010/100 | Loss 0.0060 | Win/lose count 9.0/10.893053331124904 (-1.8930533311249036)
Epoch 011/100 | Loss 0.0045 

Epoch 093/100 | Loss 0.0199 | Win/lose count 24.5/1.691610547190077 (22.808389452809923)
Epoch 094/100 | Loss 0.0150 | Win/lose count 17.0/1.6168156021627476 (15.383184397837253)
Epoch 095/100 | Loss 0.0137 | Win/lose count 22.5/1.7281382050248213 (20.77186179497518)
Epoch 096/100 | Loss 0.0070 | Win/lose count 22.0/1.803094212653478 (20.196905787346523)
Epoch 097/100 | Loss 0.0173 | Win/lose count 22.5/2.8388604235451327 (19.661139576454868)
Epoch 098/100 | Loss 0.0077 | Win/lose count 17.5/6.526455437958718 (10.973544562041283)
Epoch 099/100 | Loss 0.0089 | Win/lose count 18.5/2.3516584108619845 (16.148341589138017)


In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 20.0/1.0. Average score (19.0)
Win/lose count 20.5/2.0. Average score (18.75)
Win/lose count 19.5/2.0. Average score (18.333333333333332)
Win/lose count 20.0/3.0. Average score (18.0)
Win/lose count 25.0/0. Average score (19.4)
Win/lose count 23.5/0. Average score (20.083333333333332)
Win/lose count 33.5/3.0. Average score (21.571428571428573)
Win/lose count 13.0/2.0. Average score (20.25)
Win/lose count 25.0/1.0. Average score (20.666666666666668)
Win/lose count 23.0/1.0. Average score (20.8)
Final score: 20.8


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

I did not do the bonus question.

***